# GUI-Based Prediction Tool

## Installation

Before running the prediction interface, please install the required Python packages by executing the following command:

```bash
pip install numpy tensorflow opencv-python ipywidgets matplotlib pillow


In [6]:
import numpy as np
import tensorflow as tf
import cv2
import json
import pickle
from tensorflow.keras.preprocessing import image
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
from PIL import Image
import io
import base64
import os

class UrbanIssueInference:
    """Standalone inference class for urban issue detection with standardized categories"""

    def __init__(self, model_path, class_mapping_path, use_pickle=False, pickle_path=None):
        # Load model from pickle or h5 file
        if use_pickle and pickle_path and os.path.exists(pickle_path):
            print(f"🔄 Loading model from pickle: {pickle_path}")
            with open(pickle_path, 'rb') as f:
                self.model = pickle.load(f)
        else:
            print(f"🔄 Loading model from h5: {model_path}")
            self.model = tf.keras.models.load_model(model_path)
            
            # Option to save as pickle for faster loading next time
            if pickle_path:
                self.save_model_as_pickle(pickle_path)

        with open(class_mapping_path, 'r') as f:
            class_mapping = json.load(f)

        self.class_names = class_mapping['class_names']
        
        # Define standardized issue categories
        self.standardized_issues = [
            "Potholes",
            "Waterlogged Potholes", 
            "Damaged Road",
            "Fallen Trees",
            "Damaged Road Signs",
            "Vehicle Accidents",
            "Illegal Parking",
            "Littering",
            "Vandalism",
            "Exhaust Emission"
        ]
        
        # Map model predictions to standardized issues
        self.prediction_to_standard = {
            'pothole': 'Potholes',
            'road_crack': 'Damaged Road',
            'road_damage': 'Damaged Road', 
            'concrete_damage': 'Damaged Road',
            'water_issue': 'Waterlogged Potholes',
            'overgrown_branches': 'Fallen Trees',
            'sign_damage': 'Damaged Road Signs',
            'sign_damage_vandalism': 'Damaged Road Signs',
            'sign_healthy': 'Damaged Road Signs',  # Will be filtered out in post-processing
            'accident': 'Vehicle Accidents',
            'illegal_parking': 'Illegal Parking',
            'trash': 'Littering',
            'graffiti': 'Vandalism',
            'graffiti_vandalism': 'Vandalism', 
            'vehicle_exhaust': 'Exhaust Emission'
        }
        
        # Define main category mapping for standardized issues
        self.standard_to_main_category = {
            'Potholes': 'Road Issues',
            'Waterlogged Potholes': 'Road Issues',
            'Damaged Road': 'Road Issues',
            'Fallen Trees': 'Road Issues',
            'Damaged Road Signs': 'Road Issues',
            'Vehicle Accidents': 'Road Issues',
            'Illegal Parking': 'Parking Violations',
            'Littering': 'Public Cleanliness',
            'Vandalism': 'Public Cleanliness',
            'Exhaust Emission': 'Public Cleanliness'
        }
        
        print(f"✅ Loaded {len(self.class_names)} model classes")
        print(f"✅ Mapping to {len(self.standardized_issues)} standardized issues")

    def save_model_as_pickle(self, pickle_path):
        """Save the loaded model as a pickle file"""
        try:
            with open(pickle_path, 'wb') as f:
                pickle.dump(self.model, f)
            print(f"💾 Model saved as pickle: {pickle_path}")
            
            # Compare file sizes
            if hasattr(self, 'original_h5_path'):
                h5_size = os.path.getsize(self.original_h5_path) / (1024*1024)  # MB
                pickle_size = os.path.getsize(pickle_path) / (1024*1024)  # MB
                print(f"📊 H5 size: {h5_size:.1f} MB, Pickle size: {pickle_size:.1f} MB")
        except Exception as e:
            print(f"❌ Error saving pickle: {e}")

    def load_from_pickle(self, pickle_path):
        """Load model from pickle file"""
        try:
            with open(pickle_path, 'rb') as f:
                self.model = pickle.load(f)
            print(f"✅ Model loaded from pickle: {pickle_path}")
            return True
        except Exception as e:
            print(f"❌ Error loading from pickle: {e}")
            return False

    def get_standardized_issue(self, model_prediction):
        """Convert model prediction to standardized issue"""
        return self.prediction_to_standard.get(model_prediction, 'Other Issue')

    def get_main_category(self, standardized_issue):
        """Get main category for a standardized issue"""
        return self.standard_to_main_category.get(standardized_issue, 'Other')

    def predict_from_uploaded_image(self, uploaded_image_data, img_size=(224, 224), confidence_threshold=0.7):
        """
        Predict urban issue from uploaded image data
        """
        try:
            # Convert uploaded image data to PIL Image
            img = Image.open(io.BytesIO(uploaded_image_data))
            
            # Convert to RGB if necessary
            if img.mode != 'RGB':
                img = img.convert('RGB')
                
            # Resize image
            img = img.resize(img_size)
            
            # Convert to array and preprocess
            img_array = np.array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = img_array / 255.0

            # Make prediction
            predictions = self.model.predict(img_array, verbose=0)
            
            # Get the highest confidence prediction
            max_idx = np.argmax(predictions[0])
            max_confidence = predictions[0][max_idx]
            max_model_class = self.class_names[max_idx]
            max_standard_issue = self.get_standardized_issue(max_model_class)
            max_main_category = self.get_main_category(max_standard_issue)
            
            # Find all predictions above threshold
            qualifying_predictions = []
            
            for i, confidence in enumerate(predictions[0]):
                if confidence >= confidence_threshold:
                    model_class = self.class_names[i]
                    standard_issue = self.get_standardized_issue(model_class)
                    main_category = self.get_main_category(standard_issue)
                    
                    if standard_issue != 'Other Issue':
                        qualifying_predictions.append({
                            'model_prediction': model_class,
                            'subcategory': standard_issue,
                            'main_category': main_category,
                            'confidence': float(confidence),
                            'index': i
                        })
            
            qualifying_predictions.sort(key=lambda x: x['confidence'], reverse=True)
            
            # Handle cases with no predictions meeting threshold
            if not qualifying_predictions:
                if max_standard_issue != 'Other Issue':
                    final_predictions = [{
                        'model_prediction': max_model_class,
                        'subcategory': max_standard_issue,
                        'main_category': max_main_category,
                        'confidence': float(max_confidence),
                        'index': max_idx
                    }]
                    prediction_type = "fallback_highest"
                else:
                    sorted_indices = np.argsort(predictions[0])[::-1]
                    for idx in sorted_indices:
                        model_class = self.class_names[idx]
                        standard_issue = self.get_standardized_issue(model_class)
                        if standard_issue != 'Other Issue':
                            final_predictions = [{
                                'model_prediction': model_class,
                                'subcategory': standard_issue,
                                'main_category': self.get_main_category(standard_issue),
                                'confidence': float(predictions[0][idx]),
                                'index': idx
                            }]
                            prediction_type = "fallback_valid_issue"
                            break
                    else:
                        final_predictions = [{
                            'model_prediction': max_model_class,
                            'subcategory': 'No Issue Detected',
                            'main_category': 'Normal',
                            'confidence': float(max_confidence),
                            'index': max_idx
                        }]
                        prediction_type = "no_issue_detected"
            else:
                final_predictions = qualifying_predictions
                prediction_type = "threshold_met"
            
            main_prediction = final_predictions[0]
            
            # Get top 5 predictions for display
            top_5_indices = np.argsort(predictions[0])[::-1][:5]
            top_predictions = []
            for idx in top_5_indices:
                conf = predictions[0][idx]
                model_class = self.class_names[idx]
                standard_issue = self.get_standardized_issue(model_class)
                main_cat = self.get_main_category(standard_issue)
                top_predictions.append({
                    'model_class': model_class,
                    'standard_issue': standard_issue,
                    'main_category': main_cat,
                    'confidence': float(conf)
                })
            
            return {
                'success': True,
                'prediction_type': prediction_type,
                'threshold_used': confidence_threshold,
                'main_prediction': main_prediction,
                'top_predictions': top_predictions,
                'processed_image': img
            }
            
        except Exception as e:
            return {
                'success': False,
                'error': str(e)
            }

class UrbanIssueGUI:
    def __init__(self, model_path='urban_classifier_final.h5', class_mapping_path='class_mapping.json', 
                 use_pickle=False, pickle_path='best_model.pkl'):
        # Try to initialize the detector
        try:
            self.detector = UrbanIssueInference(
                model_path, 
                class_mapping_path, 
                use_pickle=use_pickle, 
                pickle_path=pickle_path
            )
            self.model_loaded = True
        except Exception as e:
            print(f"⚠️ Could not load model: {e}")
            print("📝 Please make sure model files are in your notebook directory")
            self.model_loaded = False
        
        self.setup_interface()
    
    def setup_interface(self):
        """Set up the web-like interface"""
        
        # Custom CSS for better styling
        display(HTML("""
        <style>
        .upload-area {
            border: 2px dashed #ccc;
            border-radius: 10px;
            padding: 20px;
            text-align: center;
            background-color: #f9f9f9;
            margin: 10px 0;
        }
        .results-container {
            background-color: #f0f8ff;
            border-radius: 10px;
            padding: 15px;
            margin: 10px 0;
            border-left: 4px solid #4CAF50;
        }
        .prediction-card {
            background-color: white;
            border-radius: 8px;
            padding: 10px;
            margin: 5px 0;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
        .confidence-bar {
            background-color: #e0e0e0;
            border-radius: 10px;
            overflow: hidden;
            height: 20px;
            margin: 5px 0;
        }
        .confidence-fill {
            height: 100%;
            background: linear-gradient(90deg, #4CAF50, #8BC34A);
            border-radius: 10px;
        }
        </style>
        """))
        
        # Title
        title = widgets.HTML("""
        <h1 style='color: #2E86C1; text-align: center; font-family: Arial, sans-serif;'>
        🏙️ Urban Issue Detection System
        </h1>
        <p style='text-align: center; color: #666; font-size: 16px;'>
        Upload an image to detect urban issues like potholes, damaged roads, littering, and more!
        </p>
        """)
        
        # File upload widget
        self.upload_widget = widgets.FileUpload(
            accept='image/*',
            multiple=False,
            description='📁 Choose Image',
            style={'button_color': '#4CAF50'}
        )
        
        # Predict button
        self.predict_button = widgets.Button(
            description='🔍 Analyze Image',
            button_style='success',
            layout=widgets.Layout(width='200px', height='40px'),
            disabled=not self.model_loaded
        )
        
        # Save pickle button
        self.save_pickle_button = widgets.Button(
            description='💾 Save as Pickle',
            button_style='info',
            layout=widgets.Layout(width='200px', height='40px'),
            disabled=not self.model_loaded
        )
        
        # Output areas
        self.image_output = widgets.Output()
        self.results_output = widgets.Output()
        
        # Event handlers
        self.upload_widget.observe(self.on_upload, names='value')
        self.predict_button.on_click(self.on_predict)
        self.save_pickle_button.on_click(self.on_save_pickle)
        
        # Layout
        upload_box = widgets.VBox([
            widgets.HTML('<div class="upload-area">'),
            self.upload_widget,
            widgets.HTML('</div>')
        ])
        
        controls = widgets.HBox([
            self.predict_button,
            self.save_pickle_button
        ], layout=widgets.Layout(justify_content='center', align_items='center'))
        
        # Status message
        if not self.model_loaded:
            status_msg = widgets.HTML("""
            <div style='background-color: #fff3cd; border: 1px solid #ffeaa7; 
                        border-radius: 5px; padding: 15px; margin: 10px 0; text-align: center;'>
                <strong>⚠️ Model not loaded!</strong><br>
                Please ensure model files are in your notebook directory.
            </div>
            """)
        else:
            status_msg = widgets.HTML("""
            <div style='background-color: #d4edda; border: 1px solid #c3e6cb; 
                        border-radius: 5px; padding: 10px; margin: 10px 0; text-align: center;'>
                <strong>✅ Model loaded successfully!</strong> Ready to analyze images.
            </div>
            """)
        
        # Main interface
        self.interface = widgets.VBox([
            title,
            status_msg,
            upload_box,
            controls,
            self.image_output,
            self.results_output
        ])
        
        # Show categories info
        if self.model_loaded:
            self.show_categories_info()
    
    def on_save_pickle(self, button):
        """Handle save pickle button click"""
        if not self.model_loaded:
            return
        
        with self.results_output:
            clear_output(wait=True)
            display(widgets.HTML("""
            <div style='text-align: center; padding: 20px;'>
                <h3>💾 Saving model as pickle...</h3>
                <p>This may take a moment.</p>
            </div>
            """))
        
        try:
            # Save the model as pickle
            pickle_filename = 'best_model.pkl'
            self.detector.save_model_as_pickle(pickle_filename)
            
            with self.results_output:
                clear_output(wait=True)
                display(widgets.HTML(f"""
                <div style='background-color: #d4edda; border: 1px solid #c3e6cb; 
                            border-radius: 5px; padding: 15px; text-align: center; color: #155724;'>
                    <strong>✅ Model saved successfully!</strong><br>
                    Saved as: {pickle_filename}<br>
                    Next time you can load this pickle file for faster startup.
                </div>
                """))
        except Exception as e:
            with self.results_output:
                clear_output(wait=True)
                display(widgets.HTML(f"""
                <div style='background-color: #f8d7da; border: 1px solid #f5c6cb; 
                            border-radius: 5px; padding: 15px; text-align: center; color: #721c24;'>
                    <strong>❌ Error saving pickle:</strong><br>{str(e)}
                </div>
                """))
    
    def show_categories_info(self):
        """Display available categories"""
        categories_html = """
        <div style='background-color: #e8f5e8; border-radius: 10px; padding: 15px; margin: 20px 0;'>
        <h3 style='color: #2E86C1; margin-top: 0;'>📋 Detectable Urban Issues:</h3>
        <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 10px;'>
        """
        
        # Group by main categories
        main_categories = {}
        for issue in self.detector.standardized_issues:
            main_cat = self.detector.get_main_category(issue)
            if main_cat not in main_categories:
                main_categories[main_cat] = []
            main_categories[main_cat].append(issue)
        
        colors = {'Road Issues': '#FF6B6B', 'Parking Violations': '#4ECDC4', 'Public Cleanliness': '#45B7D1'}
        
        for main_cat, issues in main_categories.items():
            color = colors.get(main_cat, '#95A5A6')
            categories_html += f"""
            <div style='background-color: white; border-radius: 8px; padding: 10px; border-left: 4px solid {color};'>
                <h4 style='margin: 0 0 8px 0; color: {color};'>{main_cat}</h4>
                <ul style='margin: 0; padding-left: 20px; color: #555;'>
            """
            for issue in issues:
                categories_html += f"<li>{issue}</li>"
            categories_html += "</ul></div>"
        
        categories_html += "</div></div>"
        
        categories_widget = widgets.HTML(categories_html)
        self.interface.children = self.interface.children + (categories_widget,)
    
    def on_upload(self, change):
        """Handle file upload"""
        if change['new']:
            with self.image_output:
                clear_output(wait=True)
                print("📁 Image uploaded successfully! Click 'Analyze Image' to start prediction.")
    
    def on_predict(self, button):
        """Handle prediction button click"""
        if not self.upload_widget.value:
            with self.results_output:
                clear_output(wait=True)
                display(widgets.HTML("""
                <div style='background-color: #f8d7da; border: 1px solid #f5c6cb; 
                            border-radius: 5px; padding: 15px; text-align: center; color: #721c24;'>
                    <strong>⚠️ No image selected!</strong><br>Please upload an image first.
                </div>
                """))
            return
        
        if not self.model_loaded:
            with self.results_output:
                clear_output(wait=True)
                display(widgets.HTML("""
                <div style='background-color: #f8d7da; border: 1px solid #f5c6cb; 
                            border-radius: 5px; padding: 15px; text-align: center; color: #721c24;'>
                    <strong>❌ Model not loaded!</strong><br>Please check your model files.
                </div>
                """))
            return
        
        # Show loading message
        with self.results_output:
            clear_output(wait=True)
            display(widgets.HTML("""
            <div style='text-align: center; padding: 20px;'>
                <h3>🔄 Analyzing image...</h3>
                <p>Please wait while we process your image.</p>
            </div>
            """))
        
        # Get uploaded image - handle both tuple and dict formats
        try:
            if isinstance(self.upload_widget.value, tuple):
                # New format: tuple of FileUpload objects
                uploaded_file = self.upload_widget.value[0]
                image_data = uploaded_file.content
                filename = uploaded_file.name
            else:
                # Old format: dictionary
                uploaded_file = list(self.upload_widget.value.values())[0]
                image_data = uploaded_file['content']
                filename = uploaded_file['metadata']['name']
        except Exception as e:
            with self.results_output:
                clear_output(wait=True)
                display(widgets.HTML(f"""
                <div style='background-color: #f8d7da; border: 1px solid #f5c6cb; 
                            border-radius: 5px; padding: 15px; text-align: center; color: #721c24;'>
                    <strong>❌ Error accessing uploaded file:</strong><br>{str(e)}
                </div>
                """))
            return
        
        # Make prediction with fixed confidence threshold
        result = self.detector.predict_from_uploaded_image(
            image_data, 
            confidence_threshold=0.7  # Fixed at 70%
        )
        
        # Display results
        self.display_results(result, filename)
    
    def display_results(self, result, filename):
        """Display prediction results"""
        with self.image_output:
            clear_output(wait=True)
            
            if result['success']:
                # Display the processed image
                fig, ax = plt.subplots(1, 1, figsize=(8, 6))
                ax.imshow(result['processed_image'])
                ax.set_title(f"📷 Uploaded Image: {filename}", fontsize=14, pad=20)
                ax.axis('off')
                plt.tight_layout()
                plt.show()
        
        with self.results_output:
            clear_output(wait=True)
            
            if not result['success']:
                display(widgets.HTML(f"""
                <div style='background-color: #f8d7da; border: 1px solid #f5c6cb; 
                            border-radius: 5px; padding: 15px; text-align: center; color: #721c24;'>
                    <strong>❌ Error during prediction:</strong><br>{result['error']}
                </div>
                """))
                return
            
            main_pred = result['main_prediction']
            
            # Main result card
            confidence_percent = main_pred['confidence'] * 100
            
            # Determine result color based on confidence
            if confidence_percent >= 80:
                result_color = "#4CAF50"  # Green
                confidence_text = "High Confidence"
            elif confidence_percent >= 60:
                result_color = "#FF9800"  # Orange
                confidence_text = "Medium Confidence"
            else:
                result_color = "#F44336"  # Red
                confidence_text = "Low Confidence"
            
            main_result_html = f"""
            <div class="results-container" style="border-left-color: {result_color};">
                <h2 style="margin-top: 0; color: {result_color};">🎯 Detection Result</h2>
                <div class="prediction-card">
                    <h3 style="color: #2E86C1; margin: 0;">📌 {main_pred['subcategory']}</h3>
                    <p style="margin: 5px 0; color: #666; font-size: 16px;">
                        <strong>Category:</strong> {main_pred['main_category']}
                    </p>
                    <p style="margin: 5px 0; color: #666;">
                        <strong>Confidence:</strong> {confidence_percent:.1f}% ({confidence_text})
                    </p>
                    <div class="confidence-bar">
                        <div class="confidence-fill" style="width: {confidence_percent}%; background-color: {result_color};"></div>
                    </div>
                </div>
            </div>
            """
            
            # Top predictions - show 1 if confidence >= 70%, otherwise show 3
            num_predictions_to_show = 1 if confidence_percent >= 70 else 3
            predictions_to_show = result['top_predictions'][:num_predictions_to_show]
            
            if confidence_percent >= 70:
                section_title = "🎯 High Confidence Detection"
                section_subtitle = f"Showing top prediction (confidence ≥ 70%)"
            else:
                section_title = "🔍 Multiple Possibilities"
                section_subtitle = f"Showing top 3 predictions (confidence < 70%)"
            
            top_pred_html = f"""
            <div style='background-color: #f8f9fa; border-radius: 10px; padding: 15px; margin: 10px 0;'>
                <h3 style='color: #2E86C1; margin-top: 0;'>{section_title}</h3>
                <p style='color: #666; margin: 0 0 15px 0; font-style: italic;'>{section_subtitle}</p>
            """
            
            for i, pred in enumerate(predictions_to_show, 1):
                conf_percent = pred['confidence'] * 100
                bar_color = "#4CAF50" if i == 1 else "#FF9800" if conf_percent >= 50 else "#F44336"
                
                top_pred_html += f"""
                <div style='background-color: white; border-radius: 5px; padding: 10px; margin: 5px 0; 
                           border-left: 3px solid {bar_color};'>
                    <strong>{i}. {pred['standard_issue']}</strong> → {pred['main_category']}
                    <div style='font-size: 12px; color: #666;'>Model: {pred['model_class']}</div>
                    <div class="confidence-bar" style="height: 15px;">
                        <div style="width: {conf_percent}%; height: 100%; background-color: {bar_color}; 
                                   border-radius: 10px; display: flex; align-items: center; justify-content: center; 
                                   color: white; font-size: 11px; font-weight: bold;">
                            {conf_percent:.1f}%
                        </div>
                    </div>
                </div>
                """
            
            top_pred_html += "</div>"
            
            # Additional info
            info_html = f"""
            <div style='background-color: #e7f3ff; border-radius: 8px; padding: 12px; margin: 10px 0; 
                       border-left: 3px solid #2196F3;'>
                <p style='margin: 0; color: #1976D2;'>
                    <strong>📈 Detection Details:</strong><br>
                    • Prediction Type: {result['prediction_type'].replace('_', ' ').title()}<br>
                    • Confidence Threshold: 70% (Fixed)<br>
                    • Model Input Size: 224×224 pixels
                </p>
            </div>
            """
            
            display(widgets.HTML(main_result_html + top_pred_html + info_html))
    
    def show(self):
        """Display the interface"""
        display(self.interface)

# Utility functions for model conversion
def convert_h5_to_pickle(h5_path, pickle_path):
    """Standalone function to convert h5 model to pickle"""
    try:
        print(f"🔄 Loading model from: {h5_path}")
        model = tf.keras.models.load_model(h5_path)
        
        print(f"💾 Saving model as pickle: {pickle_path}")
        with open(pickle_path, 'wb') as f:
            pickle.dump(model, f)
        
        # Compare file sizes
        h5_size = os.path.getsize(h5_path) / (1024*1024)  # MB
        pickle_size = os.path.getsize(pickle_path) / (1024*1024)  # MB
        print(f"✅ Conversion complete!")
        print(f"📊 H5 size: {h5_size:.1f} MB, Pickle size: {pickle_size:.1f} MB")
        
    except Exception as e:
        print(f"❌ Error during conversion: {e}")

def load_model_from_pickle(pickle_path):
    """Load model from pickle file"""
    try:
        with open(pickle_path, 'rb') as f:
            model = pickle.load(f)
        print(f"✅ Model loaded from pickle: {pickle_path}")
        return model
    except Exception as e:
        print(f"❌ Error loading from pickle: {e}")
        return None

# Usage Examples:
print("🚀 Initializing Urban Issue Detection Interface with Pickle Support...")

# Method 1: Use the GUI with pickle support
# This will automatically save as pickle if pickle_path is provided
gui = UrbanIssueGUI(
    model_path='./models/urban_classifier_final.h5',
    class_mapping_path='./models/class_mapping.json',
    use_pickle=False,  # Set to True if you want to load from pickle
    pickle_path='model.pkl'
)

# Method 2: Convert your existing model to pickle manually
# Uncomment the line below to convert your h5 model to pickle
# convert_h5_to_pickle('urban_classifier_final.h5', 'best_model.pkl')

print("\n" + "="*60)
print("📱 URBAN ISSUE DETECTION INTERFACE WITH PICKLE SUPPORT")
print("="*60)
print("💡 Features:")
print("   • Load model from H5 or Pickle format")
print("   • Save model as Pickle for faster loading")
print("   • Click 'Save as Pickle' button to convert your model")
print("="*60)

gui.show()

🚀 Initializing Urban Issue Detection Interface with Pickle Support...
🔄 Loading model from h5: ./models/urban_classifier_final.h5
💾 Model saved as pickle: model.pkl
✅ Loaded 16 model classes
✅ Mapping to 10 standardized issues



📱 URBAN ISSUE DETECTION INTERFACE WITH PICKLE SUPPORT
💡 Features:
   • Load model from H5 or Pickle format
   • Save model as Pickle for faster loading
   • Click 'Save as Pickle' button to convert your model
